In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai import ChatMistralAI
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel
from typing import Optional

import pandas as pd
import re

In [ ]:
reviews = pd.read_csv('/home/antog96/Jedha_Bootcamp/Projet_DSFS/anime-recommendation-engine/data/.csv')

In [6]:
reviews.head()

,profile,anime_uid,text,score,classifications
0,DesolatePsyche,34096,[' more pics Overall 8 Story 8 Animation 8 Sou...,8,"[{'label': 'POSITIVE', 'score': 0.931263029575..."
1,baekbeans,34599,[' more pics Overall 10 Story 10 Animation 10 ...,10,"[{'label': 'POSITIVE', 'score': 0.882984995841..."
2,skrn,28891,[' more pics Overall 7 Story 7 Animation 9 Sou...,7,"[{'label': 'POSITIVE', 'score': 0.972522079944..."
3,edgewalker00,2904,[' more pics Overall 9 Story 9 Animation 9 Sou...,9,"[{'label': 'NEGATIVE', 'score': 0.999156832695..."
4,aManOfCulture99,4181,[' more pics Overall 10 Story 10 Animation 8 S...,10,"[{'label': 'POSITIVE', 'score': 0.998817861080..."


In [73]:
cowboy_bebop_reviews = reviews[reviews["anime_uid"] == 303]

In [74]:
cowboy_bebop_reviews.head()

,profile,anime_uid,text,score,classifications
81404,TheCompletionist,303,[' more pics Overall 9 Story 9 Animation 9 Sou...,9,"[{'label': 'NEGATIVE', 'score': 0.887315869331..."
81584,xXKasumiXx,303,[' more pics Overall 7 Story 6 Animation 7 Sou...,7,"[{'label': 'NEGATIVE', 'score': 0.656624197959..."
81585,MKSTEEL,303,[' more pics Overall 5 Story 5 Animation 9 Sou...,5,"[{'label': 'NEGATIVE', 'score': 0.991555511951..."
81589,Ranivus,303,[' more pics Overall 6 Story 5 Animation 7 Sou...,6,"[{'label': 'POSITIVE', 'score': 0.970171928405..."
81590,Alexiell,303,[' more pics Overall 9 Story 6 Animation 7 Sou...,9,"[{'label': 'NEGATIVE', 'score': 0.992018640041..."


In [205]:
# Output format
class OutputSchema(BaseModel):
    positive: str
    negative: str

parser = PydanticOutputParser(pydantic_object=OutputSchema)
format_instructions = parser.get_format_instructions()


# Prompt string 
sys_prompt="""
You are a specialized in analysing the positive and negative elements in reviews of animes left by different users. Your task is to provide user with a summary of reviews left by other users on the platform to help them determine whether to watch that anime or not.

Ignore the "more pics" citation at the beginning of each review, as it refers to something outside of it.

If there is no review to analyse for this anime, return "No review to summarize"

Analyze the users' sentence and extract, if they are encountered:
- the positive points expressed in the reviews,
- the negative points on which the criticism is based.

Your response shall summarize the key aspects of the anime, taking into account that there are several reviews left by as many users, but without explicitly mentioning the amount of reviews left. Please be relatively measured with the way you present negative criticism.
You shall also take into account the proportion difference between positive and negative feedbacks.

Return your response as a JSON object containing up to two fields:
If no positive feedback is provided, ignore the generation of the sentence, and proceed to the next point.
- positive: a single string summarizing the positive aspects raised, starting with "Users liked:"
If no negative aspect is mentioned, or if they are disproportionately rare compared to the positive feedback, ignore the generation of the sentence. Otherwise, proceed as follows :
- negative: a single string summarizing the negative aspects raised, starting with "Users disliked:".

If there is no review to analyse for this anime, return "This anime has no review yet! You can watch it and review it yourself!" in the JSON file for positive reviews, and "" in the JSON file for negative reviews.

{{format_instructions}}
"""

# Define system prompt
start_prompt = ChatPromptTemplate.from_messages([
    ("system", sys_prompt),
    ("user", "{text}")
])

In [ ]:
%env MISTRAL_API_KEY=

In [207]:
llm = ChatMistralAI(model="mistral-medium-latest")

In [208]:
model_llm = start_prompt | llm 

In [190]:
test_reviews = reviews[reviews["anime_uid"] == 99314]

In [209]:
test_reviews.head()

,profile,anime_uid,text,score,classifications


In [210]:
input = test_reviews['text']

In [211]:
response = model_llm.invoke({"text": input, "format_instructions": parser.get_format_instructions()})
input_positive_clean = parser.parse(response.content).positive
input_negative_clean = parser.parse(response.content).negative

if input_positive_clean:
    print(input_positive_clean)
    print(input_negative_clean)
else:
    print("Please try again with a different phrasing.")

This anime has no review yet! You can watch it and review it yourself!

